In [1]:
import math
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

In [2]:
import sys
from io import StringIO
from IPython import get_ipython


class IpyExit(SystemExit):
    """Exit Exception for IPython.

    Exception temporarily redirects stderr to buffer.
    """
    def __init__(self):
        # print("exiting")  # optionally print some message to stdout, too
        # ... or do other stuff before exit
        sys.stderr = StringIO()

    def __del__(self):
        sys.stderr.close()
        sys.stderr = sys.__stderr__  # restore from backup


def ipy_exit():
    raise IpyExit


if get_ipython():    # ...run with IPython
    exit = ipy_exit  # rebind to custom exit
else:
    exit = exit      # just make exit importable

In [3]:
from scipy import integrate
import numpy.ma as ma

In [27]:
def fibonacci_sphere(R=1.0, samples=1):

    points = []
    phi = math.pi * (3. - math.sqrt(5.))  # golden angle in radians

    for i in range(samples):
        y = R*(1 - (i / float(samples - 1)) * 2)  # y goes from 1 to -1
        radius = math.sqrt(R*R - y * y)  # radius at y

        theta = phi * i  # golden angle increment

        x = math.cos(theta) * radius
        z = math.sin(theta) * radius

        #points.append([x, y, z])
        points.append([y, x, z])

    return points

In [73]:
L=5                        #radius of condensate
R=0.25                     #RNA length/2
r_i = 0.1
r_o = 0.6#1.1
#x = np.arange(20,800,50)  #aggregate size
px = np.linspace(0.05, 0.63, 15)
Ns = 5
epsilon = 0.005
D_o=30
k_in = 2
coef = k_in/4/np.pi
lamb0 = 1+coef/D_o*(-1/(2*R))
D_in=[1,3,30]  


grid=8
g=[]

x = np.linspace(-L, L, grid, endpoint=False)+L/grid
y = np.linspace(-L, L, grid, endpoint=False)+L/grid
z = np.linspace(-L, L, grid, endpoint=False)+L/grid
xv, yv, zv = np.meshgrid(x, y, z)
meshT=[]
meshT.append(np.concatenate(xv,axis=None))
meshT.append(np.concatenate(yv,axis=None))
meshT.append(np.concatenate(zv,axis=None))
mesh = np.transpose(meshT)

for i in range(len(mesh)):
    if (mesh[i][0]**2 + mesh[i][1]**2 + mesh[i][2]**2) < (L-L/grid)**2:            #trap density inside the sphere with a radius of L is conserved
        g.append(mesh[i])

final=[]
final_std=[]


for D_i in D_in:
    mean=[]
    D_p = D_o + D_i
    D_m = D_o - D_i
    
    y2 = lambda t,a1,a2,a3,b1,b2,b3: t**(-D_o/D_p)/((a1-b1*t)**2+(a2-b2*t)**2+(a3-b3*t)**2)**(0.5)
    
    for sample in range(5):   #run 5 times to calculate statistic average and variance          
        ansN=[]

        for p in px:
            pos_B=[]
            pos_E=[]

            for i in range(len(g)):
                if np.random.rand(1) < p:
                        s1_pos_B=fibonacci_sphere(r_o,Ns)
                        s1_pos_E=fibonacci_sphere(r_i,Ns) 
                        for j in range(Ns):
                            for k in range(3):
                                s1_pos_B[j][k] += g[i][k]
                                s1_pos_E[j][k] += g[i][k]

                            pos_B.append(s1_pos_B[j])
                            pos_E.append(s1_pos_E[j]) 

            
            
            M=len(pos_B)       
            lamb=np.zeros((M,M))
            for j in range(M):          
                rk_E=L/np.sqrt(np.dot(pos_E[j],pos_E[j]))
                rk_B=L/np.sqrt(np.dot(pos_B[j],pos_B[j]))
                for k in range(M):
                    if j == k:
                        vec2 = np.subtract(pos_B[k], pos_E[j])
                        vec4 = np.subtract(pos_B[k], [rk_E**2*pos_E[j][0],rk_E**2*pos_E[j][1],rk_E**2*pos_E[j][2]])
                        lamb1 = - (1/D_i/np.sqrt(np.dot(vec2,vec2))-rk_E*D_m/D_p/D_i/np.sqrt(np.dot(vec4,vec4))) - \
                            (D_m/D_p**2)*(-rk_E*integrate.quad(y2, 1, np.inf,args=(pos_B[k][0],pos_B[k][1],pos_B[k][2],rk_E**2*pos_E[j][0],rk_E**2*pos_E[j][1],rk_E**2*pos_E[j][2]))[0])                        
                        
                        lamb[k][j] = 1+coef*lamb1
                        
                    else:
                        vec1 = np.subtract(pos_B[k], pos_B[j])
                        vec2 = np.subtract(pos_B[k], pos_E[j])
                        vec3 = np.subtract(pos_B[k], [rk_B**2*pos_B[j][0],rk_B**2*pos_B[j][1],rk_B**2*pos_B[j][2]])
                        vec4 = np.subtract(pos_B[k], [rk_E**2*pos_E[j][0],rk_E**2*pos_E[j][1],rk_E**2*pos_E[j][2]])
                        #if (np.sqrt(np.dot(vec1,vec1))<2*epsilon) or (np.sqrt(np.dot(vec2,vec2))<2*epsilon): ipy_exit() #make sure sinks/sources are not collapse with each other
                        
                        lamb[k][j] = 1/D_i/np.sqrt(np.dot(vec1,vec1))-rk_B*D_m/D_p/D_i/np.sqrt(np.dot(vec3,vec3)) - (1/D_i/np.sqrt(np.dot(vec2,vec2))-rk_E*D_m/D_p/D_i/np.sqrt(np.dot(vec4,vec4))) - \
                            (D_m/D_p**2)* \
                            (rk_B*integrate.quad(y2, 1, np.inf,args=(pos_B[k][0],pos_B[k][1],pos_B[k][2],rk_B**2*pos_B[j][0],rk_B**2*pos_B[j][1],rk_B**2*pos_B[j][2]))[0] - \
                             rk_E*integrate.quad(y2, 1, np.inf,args=(pos_B[k][0],pos_B[k][1],pos_B[k][2],rk_E**2*pos_E[j][0],rk_E**2*pos_E[j][1],rk_E**2*pos_E[j][2]))[0])                        
                        
                        lamb[k][j] = coef*lamb[k][j]
                                                
            #lambr = np.linalg.inv(lamb)  
            c0 = np.ones(M)
            ans = np.linalg.solve(lamb,c0)
            ansN.append(np.mean(ans)*lamb0)
        mean.append(ansN)

    a=np.array(mean)
    mask = np.logical_or(a == a.max(0, keepdims = 0), a == a.min(0, keepdims = 0))
    a_masked = ma.masked_array(a, mask = mask)
    
    final.append(a_masked.mean(0).data)
    final_std.append(a_masked.std(0).data)
    
np.savetxt("local_P_c.dat",final)
np.savetxt("local_P_std_c.dat",final_std)

In [74]:
L=5                        #radius of condensate
R=0.25                     #RNA length/2
r_i = 0.1
r_o = 0.6#1.1
#x = np.arange(20,800,50)  #aggregate size
px = np.linspace(0.05, 0.63, 15)
Ns = 5
epsilon = 0.005
D_o=30
k_in = 2
coef = k_in/4/np.pi
lamb0 = 1+coef/D_o*(-1/(2*R))
D_in=[1,3,30]  


grid=8
g=[]

x = np.linspace(-L, L, grid, endpoint=False)+L/grid
y = np.linspace(-L, L, grid, endpoint=False)+L/grid
z = np.linspace(-L, L, grid, endpoint=False)+L/grid
xv, yv, zv = np.meshgrid(x, y, z)
meshT=[]
meshT.append(np.concatenate(xv,axis=None))
meshT.append(np.concatenate(yv,axis=None))
meshT.append(np.concatenate(zv,axis=None))
mesh = np.transpose(meshT)

for i in range(len(mesh)):
    if (mesh[i][0]**2 + mesh[i][1]**2 + mesh[i][2]**2) < (L-L/grid)**2:            #trap density inside the sphere with a radius of L is conserved
        g.append(mesh[i])

final=[]
final_std=[]


for D_i in D_in:
    mean=[]
    D_p = D_o + D_i
    D_m = D_o - D_i
    
    y2 = lambda t,a1,a2,a3,b1,b2,b3: t**(-D_o/D_p)/((a1-b1*t)**2+(a2-b2*t)**2+(a3-b3*t)**2)**(0.5)
    
    for sample in range(5):   #run 5 times to calculate statistic average and variance          
        ansN=[]

        for p in px:
            pos_B=[]
            pos_E=[]

            for i in range(len(g)):
                if np.random.rand(1) < p:
                        s1_pos_B=fibonacci_sphere(r_i,Ns)
                        s1_pos_E=fibonacci_sphere(r_o,Ns) 
                        for j in range(Ns):
                            for k in range(3):
                                s1_pos_B[j][k] += g[i][k]
                                s1_pos_E[j][k] += g[i][k]

                            pos_B.append(s1_pos_B[j])
                            pos_E.append(s1_pos_E[j]) 

            
            
            M=len(pos_B)       
            lamb=np.zeros((M,M))
            for j in range(M):          
                rk_E=L/np.sqrt(np.dot(pos_E[j],pos_E[j]))
                rk_B=L/np.sqrt(np.dot(pos_B[j],pos_B[j]))
                for k in range(M):
                    if j == k:
                        vec2 = np.subtract(pos_B[k], pos_E[j])
                        vec4 = np.subtract(pos_B[k], [rk_E**2*pos_E[j][0],rk_E**2*pos_E[j][1],rk_E**2*pos_E[j][2]])
                        lamb1 = - (1/D_i/np.sqrt(np.dot(vec2,vec2))-rk_E*D_m/D_p/D_i/np.sqrt(np.dot(vec4,vec4))) - \
                            (D_m/D_p**2)*(-rk_E*integrate.quad(y2, 1, np.inf,args=(pos_B[k][0],pos_B[k][1],pos_B[k][2],rk_E**2*pos_E[j][0],rk_E**2*pos_E[j][1],rk_E**2*pos_E[j][2]))[0])                        
                        
                        lamb[k][j] = 1+coef*lamb1
                        
                    else:
                        vec1 = np.subtract(pos_B[k], pos_B[j])
                        vec2 = np.subtract(pos_B[k], pos_E[j])
                        vec3 = np.subtract(pos_B[k], [rk_B**2*pos_B[j][0],rk_B**2*pos_B[j][1],rk_B**2*pos_B[j][2]])
                        vec4 = np.subtract(pos_B[k], [rk_E**2*pos_E[j][0],rk_E**2*pos_E[j][1],rk_E**2*pos_E[j][2]])
                        #if (np.sqrt(np.dot(vec1,vec1))<2*epsilon) or (np.sqrt(np.dot(vec2,vec2))<2*epsilon): ipy_exit() #make sure sinks/sources are not collapse with each other
                        
                        lamb[k][j] = 1/D_i/np.sqrt(np.dot(vec1,vec1))-rk_B*D_m/D_p/D_i/np.sqrt(np.dot(vec3,vec3)) - (1/D_i/np.sqrt(np.dot(vec2,vec2))-rk_E*D_m/D_p/D_i/np.sqrt(np.dot(vec4,vec4))) - \
                            (D_m/D_p**2)* \
                            (rk_B*integrate.quad(y2, 1, np.inf,args=(pos_B[k][0],pos_B[k][1],pos_B[k][2],rk_B**2*pos_B[j][0],rk_B**2*pos_B[j][1],rk_B**2*pos_B[j][2]))[0] - \
                             rk_E*integrate.quad(y2, 1, np.inf,args=(pos_B[k][0],pos_B[k][1],pos_B[k][2],rk_E**2*pos_E[j][0],rk_E**2*pos_E[j][1],rk_E**2*pos_E[j][2]))[0])                        
                        
                        lamb[k][j] = coef*lamb[k][j]
                                                
  
            c0 = np.ones(M)
            ans = np.linalg.solve(lamb,c0)
            ansN.append(np.mean(ans)*lamb0)
        mean.append(ansN)

    a=np.array(mean)
    mask = np.logical_or(a == a.max(0, keepdims = 0), a == a.min(0, keepdims = 0))
    a_masked = ma.masked_array(a, mask = mask)
    
    final.append(a_masked.mean(0).data)
    final_std.append(a_masked.std(0).data)
    
np.savetxt("local_P_r.dat",final)
np.savetxt("local_P_std_r.dat",final_std)